In [1]:
# import libraries
import pandas as pd
import requests
import os
import json
import csv
import time
from dotenv import load_dotenv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata

In [2]:
# load environment variables in the .env file in the same directory
# You need to have .env file and set BEARER_TOKEN='PUT TWITTER BEARER TOKEN HERE'
load_dotenv()

True

In [3]:
# set headers for API request
def create_headers():
    bearer_token = os.environ.get("BEARER_TOKEN")
    return {"Authorization": "Bearer {}".format(bearer_token)}

In [4]:
# Function to generate URL and Parameters for API call
def create_url(keyword, start_time, end_time, max_results=10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    
    query_params = {
        'query': keyword,
        'start_time': start_time,
        'end_time': end_time,
        'max_results': max_results,
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        #'place.fields': 'full_name,country',
        'next_token':{}
    }
    return (search_url, query_params)

In [5]:
# Function to make GET request to desired endpoint
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token
    response = requests.request("GET", url, params=params, headers=headers)
        
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [31]:
#Inputs for the request
bearer_token = os.environ.get("BEARER_TOKEN")
headers = create_headers()
keyword = "ウーバーイーツ"
start_time = "2021-12-26T00:00:00.000Z"
end_time = "2021-12-26T00:00:30:00.000Z"
max_results = 15

In [33]:
# Call API
url = create_url(keyword, start_time, end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])


Endpoint Response Code: 401


Exception: (401, '{\n  "title": "Unauthorized",\n  "type": "about:blank",\n  "status": 401,\n  "detail": "Unauthorized"\n}')

In [9]:
# Create output csv file
csvFile = open("result.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

In [10]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that
        
        
        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "
        
        # 4. Tweet ID
        tweet_id = tweet['id']
        
        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']
        
        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        #res = [tweet_id, text]
        res = [author_id, created_at, geo, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [24]:
# Pagination for tweets over results > 100
total_tweets = 0
max_results = 100

count = 0
max_count = 100
flag = True
next_token = None


while flag:
    if count >= max_count:
        break
    print("-----------------")
    print("Token: ", next_token)
    print(count)
    url = create_url(keyword, start_time, end_time, max_results)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    result_count = json_response['meta']['result_count']
    print("INITIAL")
    
    if 'next_token' in json_response['meta']:
        # save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
        if result_count is not None and result_count > 0 and next_token is not None:
            append_to_csv(json_response, "result.csv")
            print("TYPE 1")
       
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-----------------")
            time.sleep(5)
    # if no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-----------------")
            append_to_csv(json_response, "result.csv")
            print("TYPE 2")
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-----------------")
            time.sleep(5)
            
        flag = False
        next_token = None
    time.sleep(5)
print("Total number of results: " , total_tweets)

-----------------
Token:  None
0
Endpoint Response Code: 200
INITIAL
Next Token:  b26v89c19zqg8o3fpe1715j0vj89fojfl3yxvylm3aet9
# of Tweets added from this response:  100
TYPE 1
Total # of Tweets added:  100
-----------------
-----------------
Token:  b26v89c19zqg8o3fpe1715j0vj89fojfl3yxvylm3aet9
0
Endpoint Response Code: 200
INITIAL
-----------------
# of Tweets added from this response:  28
TYPE 2
Total # of Tweets added:  128
-----------------
Total number of results:  128


In [25]:
pd.read_csv("result.csv")

,1392367096055488515,2021-12-26 00:29:52+00:00,,1474900232475492357,ja,0,0.1,0.2,0.3,Botbird tweets,配達の単価が下がったって聞きましたけど、大丈夫ですか？ もし貴方が介護の資格をもっていてデリバリーでは収入が安定しないならもう１件、訪問介護で働きません？堺市の平均時給より高いですよ、うち https://t.co/pD7EBHLyne #配達員 #ウーバーイーツ #介護 #大阪 #ヘルパー募集
0,1411142914663600129,2021-12-26 00:29:38+00:00,,1474900174115950599,ja,0,0,0,0,Botbird tweets,ウーバーイーツで初回配達＆登録をする人はいませんか？\n招待コード【pbyh6i】を入力すれ...
1,1357545037119119360,2021-12-26 00:29:30+00:00,,1474900140397940740,ja,0,0,0,0,Botbird tweets,お腹が空いていませんか？\n\n【eats-ebbk6r】 \n\nこちらのコード使うと、ウ...
2,82841493,2021-12-26 00:28:59+00:00,,1474900008575340547,ja,0,0,0,0,get-access-token-rickbox,2500円引き♪\n\n【eats-ckvsvberue】\n\n上記プロモーションコード使...
3,72681507,2021-12-26 00:28:47+00:00,,1474899961804648448,ja,0,0,0,59,Twitter Web App,RT @oscarnoyukue: 「東京自転車節」を観る。コロナ禍に上京してウーバーイーツ...
4,1455881867111002119,2021-12-26 00:28:32+00:00,,1474899896541282304,ja,0,0,0,0,twittbot.net,好きな時間で自由に休憩〜ウーバーイーツ配達員を始める方！招待コード「ag31nnsvpvfe...
5,1451181307183046660,2021-12-26 00:28:30+00:00,,1474899889662611457,ja,0,0,0,0,twittbot.net,食欲の冬！ウーバーイーツデビュー！！\nUber Eats初回限定クーポン発行中！\n\n✅...
6,1613287687,2021-12-26 00:28:30+00:00,,1474899887942946820,ja,0,0,0,0,twittbot.net,【先着5枚】ｷﾀーー(ﾟ∀ﾟ)ーー！\nUber Eatsの初回注文↓のコードで4000円割...
7,1244066887870234624,2021-12-26 00:28:23+00:00,,1474899859895644160,ja,0,0,0,0,twittbot.net,か\n友達招待コード クーポン 割引券\nプロモーションコード\nメルカリ「FPXFFY」\...
8,1256740634326515712,2021-12-26 00:28:11+00:00,,1474899810474139650,ja,0,0,0,0,twittbot.net,検索から見つけてくださりありがとうございます☆\n以下、私の招待コードです！\n\nウーバー...
9,1401894100412952578,2021-12-26 00:28:07+00:00,,1474899792212144128,ja,0,0,0,0,twittbot.net,・UberEats限定クーポン2021年6月最新版・\n\n【eats-8yi0a9ufx1...
